In [11]:
import pandas as pd

input_file = '0. Scraped.csv'
df_to_clean = pd.read_csv(input_file)
df_to_clean.columns

Index(['Unnamed: 0', 'Category Name', 'Recipe Name', 'Recipe URL',
       'Total Time', 'Rating', 'Comment', 'total_mins', 'num_ingredients',
       'Ingredients_List'],
      dtype='object')

In [12]:
# import file and create DF
input_file = '0. Scraped.csv'

df_to_clean = pd.read_csv(input_file)

In [13]:
# Create a DF of just REcipe Name and Ingreidents for easier interation
ingredient_list_df = pd.DataFrame(df_to_clean.groupby('Recipe Name')["Recipe Name", 'Ingredients_List'].apply(lambda x: x.iloc[0]))
ingredient_list_df


/var/folders/m2/b8l4_05d1p39hmx00f4y_czr0000gn/T/ipykernel_99466/1019327333.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ingredient_list_df = pd.DataFrame(df_to_clean.groupby('Recipe Name')["Recipe Name", 'Ingredients_List'].apply(lambda x: x.iloc[0]))


,Recipe Name,Ingredients_List
Recipe Name,,
"15-Minute Butter Gnocchi with Spicy Chili Crisp, Capers and Parmesan",15-Minute Butter Gnocchi with Spicy Chili Cris...,"['2 (17.6 oz) packages gnocchi', '6 tablespoon..."
3-Ingredient Banana Oatmeal Cookies,3-Ingredient Banana Oatmeal Cookies,"['3/4 cup mashed banana', '1/2 cup rolled oats..."
3-Ingredient Peanut Butter Oatmeal Cookies,3-Ingredient Peanut Butter Oatmeal Cookies,"['1/2 cup peanut butter', '6 tablespoons maple..."
4 Ingredient Lemon Pound Cake,4 Ingredient Lemon Pound Cake,"['3 large eggs', '1 (14 ounce) can sweetened c..."
4-Ingredient Keto Peanut Butter Cookies,4-Ingredient Keto Peanut Butter Cookies,"['1 cup peanut butter', '½ cup low-calorie nat..."
...,...,...
Yummy Vegan Pesto,Yummy Vegan Pesto,"['⅓ cup pine nuts', '⅔ cup olive oil', '5 clov..."
Zucchini Coffee Cake,Zucchini Coffee Cake,"['1 zucchini (10 ounces), ends trimmed', '2 cu..."
Zucchini Pizza Bites,Zucchini Pizza Bites,"['2 zucchini, sliced into ¼-inch thick rounds'..."


In [17]:
import re
import ast  # To safely evaluate the list of ingredients from a string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
common_units = {'cup', 'tablespoon', 'teaspoon', 'ounce', 'pound', 'gram', 
                'kilogram', 'ml', 'l'}
stop_words.update(common_units)

# Add verbs and other descriptors to remove
descriptors = [
    'freshly', 'grated', 'shredded', 'boneless', 'skinless', 'divided', 
    'seasoned', 'for garnish', 'more for serving', 'finely', 'minced', 
    'coarsely', 'salted', 'unsalted', 'mashed', 'diced', 'chopped', 
    'sliced', 'cubed', 'peeled', 'crushed', 'roasted', 'fried', 'baked'
]

# Function to clean and preprocess individual ingredients
def clean_and_preprocess_ingredient(ingredient):
    # Step 1: Lowercase the ingredient to standardize
    ingredient = ingredient.lower()

    # Step 2: Remove anything inside parentheses
    ingredient = re.sub(r'\(.*?\)', '', ingredient)

    # Step 3: Remove measurements, units, and amounts
    ingredient = re.sub(r'\b\d+([\/.,]?\d+)?\s*(ounces?|oz|packages?|cups?|teaspoons?|tablespoons?|pounds?|grams?|cloves?|pinches?|sticks?|slices?|sprigs?|large|small|medium)\b', '', ingredient)

    # Step 4: Remove descriptors and verbs
    for desc in descriptors:
        ingredient = ingredient.replace(desc, '')

    # Step 5: Remove punctuation and extra spaces
    ingredient = re.sub(r'[^a-z\s]', '', ingredient)
    ingredient = re.sub(r'\s+', ' ', ingredient).strip()

    # Step 6: Apply common ingredient simplifications
    common_simplifications = {
        'mozzarella cheese': 'mozzarella',
        'parmesan cheese': 'parmesan',
        'breadcrumbs': 'breadcrumbs',
        'chicken breasts': 'chicken',
        'marinara sauce': 'marinara',
        'egg whites': 'eggs',
        'cake flour': 'flour',
        'confectioners sugar': 'sugar',
        'raisins': 'raisin',
        'oats': 'oat',
        'butter': 'butter',
        'capers in brine': 'capers'
    }
    for key, value in common_simplifications.items():
        if key in ingredient:
            ingredient = value

    # Step 7: Tokenize, lemmatize, and remove stopwords
    tokens = word_tokenize(ingredient)
    lemmatized = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha()]
    final_tokens = [word for word in lemmatized if word not in stop_words]

    # Rejoin tokens into a cleaned string
    return ' '.join(final_tokens)

# Function to clean and preprocess an entire ingredient list for each recipe
def clean_and_preprocess_recipe_ingredients(ingredient_list):
    if isinstance(ingredient_list, str):
        ingredient_list = ast.literal_eval(ingredient_list)
    return [clean_and_preprocess_ingredient(ingredient) for ingredient in ingredient_list]

# Apply the function to the 'Ingredients_List' column and create a new 'Clean_Ingredient_List' column
ingredient_list_df['Clean_Ingredient_List'] = ingredient_list_df['Ingredients_List'].apply(clean_and_preprocess_recipe_ingredients)



In [18]:
# Rename columns and reorder them for export
ingredient_list_df = ingredient_list_df.rename(columns={
    'Recipe Name': 'Name', 
    'Clean_Ingredient_List': 'Ingredients'
})

# Reorder the columns
ingredient_list_df = ingredient_list_df[['Name', 'Ingredients']]

# Export to CSV with the correct filename
ingredient_list_df.to_csv('1. Recipe_Ingredients.csv', index=False)

In [8]:
import nltk
from nltk.corpus import stopwords
import string
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


# Download NLTK stopwords if not already downloaded
nltk.download('punkt')
nltk.download('stopwords') # Required for tokenization

# Step 1: Select relevant columns and rename them
grouped_df = df_to_clean[['Recipe Name', 'total_mins', 'Comment', 'Rating', 'Category Name', 'Recipe URL']].rename(
    columns={
        'Recipe Name': 'name',
        'total_mins': 'cook_time',
        'Comment': 'review',
        'Rating': 'rating',
        'Category Name': 'category',
        'Recipe URL': 'url'
    }
)

# Step 2: Define a function to clean reviews
def clean_review(text):
    if not isinstance(text, str) or text.strip() == '':  # Handle missing/empty reviews
        return ''

    # Define stopwords and punctuation
    stop_words = set(stopwords.words('english'))
    punctuations = string.punctuation

    # Tokenize the text
    words = nltk.word_tokenize(text.lower())

    # Remove stopwords and punctuation
    cleaned_words = [word for word in words if word not in stop_words and word not in punctuations]

    # Join the words back into a cleaned string
    return ' '.join(cleaned_words)

# Step 3: Apply the cleaning function to the 'review' column
grouped_df['review'] = grouped_df['review'].apply(clean_review)

# Step 4: Display the cleaned dataframe
print(grouped_df.head())

# Step 5: Save the cleaned data to a CSV file
grouped_df.to_csv("1. Recipe_Reviews.csv", index=False)


[nltk_data] Downloading package punkt to /Users/evanhadd/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/evanhadd/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                          name  cook_time  \
0   Air Fryer Chicken Parmesan       40.0   
1   Air Fryer Chicken Parmesan       40.0   
2  Chocolate Angel Food Cake I       90.0   
3  Chocolate Angel Food Cake I       90.0   
4  Chocolate Angel Food Cake I       90.0   

                                              review  rating  \
0  recently tried air fryer chicken parmesan life...     4.5   
1  recently tried air fryer chicken parmesan life...     4.5   
2  excellent angel food cake satisfying chocolate...     4.7   
3  yummy 've made twice times huge success textur...     4.7   
4  super easy recipe follow simply separate eggs ...     4.7   

            category                                                url  
0  Air Fryer Recipes  https://www.allrecipes.com/air-fryer-chicken-p...  
1  Air Fryer Recipes  https://www.allrecipes.com/air-fryer-chicken-p...  
2   Angel Food Cakes  https://www.allrecipes.com/recipe/8252/chocola...  
3   Angel Food Cakes  https://www.allrecipes.com